In [13]:
import mhdb
from mhdb import *
from mhdb.spreadsheet_io import *
from mhdb.write_rdf import *
import pandas as pd

In [8]:
infile = "/Users/jon.clucas/Downloads/behaviors-thoughts.csv"
outfile = "/Users/jon.clucas/Downloads/behaviors_thoughts.ttl"
chart = pd.read_csv(infile, index_col="Id")
chart = chart.loc[chart["Name"].isin(["Entity", "Attribute", "Line", "Rectangle"])]

In [9]:
chart

,Name,Shape Library,Page ID,Contained By,Line Source,Line Destination,Source Arrow,Destination Arrow,Text Area 1
Id,,,,,,,,,
2,Rectangle,Geometric Shapes,1.0,NaN,NaN,NaN,NaN,NaN,social-emotional reciprocity
3,Rectangle,Geometric Shapes,1.0,NaN,NaN,NaN,NaN,NaN,social approach
4,Rectangle,Geometric Shapes,1.0,NaN,NaN,NaN,NaN,NaN,normal back and forth conversation
5,Rectangle,Geometric Shapes,1.0,NaN,NaN,NaN,NaN,NaN,sharing of
6,Rectangle,Geometric Shapes,1.0,NaN,NaN,NaN,NaN,NaN,interests
7,Rectangle,Geometric Shapes,1.0,NaN,NaN,NaN,NaN,NaN,emotions/affect
8,Rectangle,Geometric Shapes,1.0,NaN,NaN,NaN,NaN,NaN,initiating social interaction
9,Rectangle,Geometric Shapes,1.0,NaN,NaN,NaN,NaN,NaN,social imitation
10,Rectangle,Geometric Shapes,1.0,NaN,NaN,NaN,NaN,NaN,responsiveness to emotions and affect


In [10]:
def rdfstring(label, superclasses, entity_type):
    
    rdf_string = """
### {0}
:{1} rdf:type :{3} ;
    rdfs:label \"{2}\" """.format(
        label,
        mhdb.spreadsheet_io.convert_string_to_label(label),
        label,
        "owl:Class"
    )
    for superclass in superclasses:
        rdf_string += """;
    rdfs:subClassOf :{0} """.format(superclass)
    return("{0} .".format(rdf_string))

In [14]:
rdf = mhdb.write_rdf.print_header(
    "http://mentalhealth.tech/behaviors_thoughts",
    "0.0.1",
    "behaviors/thoughts",
    "behaviors/thoughts")

for row in chart.iterrows():
    if(row[1].Name in ["Entity", "Attribute", "Rectangle"]):
        supers = ([*[mhdb.spreadsheet_io.convert_string_to_label(
            chart.loc[chart.index == x]["Text Area 1"].values[0]) for x in chart.loc[
            (chart["Line Source"] == row[1].name)
             &
             ("Arrow" in chart["Source Arrow"])
        ]["Line Destination"]], *[mhdb.spreadsheet_io.convert_string_to_label(
            chart.loc[chart.index == x]["Text Area 1"].values[0]) for x in chart.loc[
            (chart["Line Destination"] == row[1].name)
             &
             ("Arrow" in chart["Destination Arrow"])
        ]["Line Source"]]])
        rdf += rdfstring(row[1]["Text Area 1"], supers, row[1].Name)
        
with open(outfile, "w") as o:
    o.write(rdf)